In [ ]:
#!pip install evaluate

In [ ]:
#!pip install transformers

In [ ]:
#!pip install datasets

In [1]:
import tensorflow as tf
import pandas as pd
import evaluate
import os
dir_root = os.getcwd()

C:\ProgramData\Anaconda3\envs\tfenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import TFAutoModelForSequenceClassification, TFDistilBertForSequenceClassification
tf_model = TFAutoModelForSequenceClassification.from_pretrained('./models/pretrainedModel', num_labels=6)
# tf_model.load_weights('./models/pretrainedModel/tf_model.h5')

Some layers from the model checkpoint at ./models/pretrainedModel were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./models/pretrainedModel and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
tf_model.compile(optimizer='adam', metrics = ['accuracy'], loss=loss)

In [5]:
testcsv = os.path.join(dir_root, 'data/interim/testsethugf.csv')

In [4]:
labels_set = {'ROBO A DOMICILIO',
 'ROBO A PERSONAS',
 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO DE CARROS',
 'ROBO DE MOTOS'}

In [8]:
from datasets import load_dataset, Features, ClassLabel, Value

class_names = list(labels_set)
robo_features = Features({'relato': Value('string'), 'labels': ClassLabel(names=class_names)})
dataset = load_dataset("csv", data_files={'test': testcsv, }, features=robo_features)

Using custom data configuration default-bfb2595bb00ed16d
Reusing dataset csv (C:\Users\entea\.cache\huggingface\datasets\csv\default-bfb2595bb00ed16d\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 182.59it/s]


In [9]:
from transformers import DistilBertTokenizerFast
model_name = 'distilbert-base-multilingual-cased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

In [10]:
def tokenizer_func(examples):
  return tokenizer(examples["relato"],
                   max_length=300,
                   padding = "max_length",
                   truncation=True)

tokenized_dataset = dataset.map(tokenizer_func, batched=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:17<00:00,  5.21ba/s]


In [11]:
len(tokenized_dataset['test'][-1]['input_ids']), tokenized_dataset['test'][-1]['labels']

(300, 2)

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [14]:
tf_test_set = tokenized_dataset["test"].shuffle(seed=42).select(range(4000)).to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

Loading cached shuffled indices for dataset at C:\Users\entea\.cache\huggingface\datasets\csv\default-bfb2595bb00ed16d\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58\cache-885b6e3923f270bb.arrow


In [17]:
tf_model.evaluate(tf_test_set)

250/250 [==============================] - 131s 506ms/step - loss: 4.1905 - accuracy: 0.0452


[4.190496444702148, 0.045249998569488525]

The performance has nothing to do with what was trained. I wonder if I changed the model to distilbert we had the same performance

In [18]:
model_notrainig = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['dropout_39', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream ta

In [19]:
model_notrainig.compile(optimizer='adam', metrics = ['accuracy'], loss=loss)
model_notrainig.evaluate(tf_test_set)

250/250 [==============================] - 218s 859ms/step - loss: 1.7801 - accuracy: 0.1953


[1.780104637145996, 0.195250004529953]

IT seems that the real performance of the model is below 0.46

Trying to load the model from weights

In [20]:
model_2 = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['dropout_59', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream ta

In [ ]:
# model2 = model_2.load_weights('models/tfmodelnlph5.h5')

In [ ]:
#model2w

In [ ]:
# model_2.summary()

In [ ]:
# model_2.compile(optimizer='adam', metrics = ['accuracy'], loss=loss)
model_2.compile(optimizer='adam', metrics = ['accuracy'], loss=loss)

In [ ]:
model_2.load_weights('models/tfmodelnlph5.h5')

In [ ]:
model_2.summary()

In [ ]:
model_2.evaluate(tf_test_set)